<a href="https://colab.research.google.com/github/deepenmi/Machine-learning-templates/blob/master/Autogluon_HandsOn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!python3 -m pip install -U pip


     |████████████████████████████████| 1.6MB 4.0MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [4]:
!python3 -m pip install -U setuptools wheel

In [5]:
!python3 -m pip install -U "mxnet<2.0.0"

     |████████████████████████████████| 46.9 MB 59 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [9]:
pip install autogluon


     |████████████████████████████████| 48 kB 1.7 MB/s 
     |████████████████████████████████| 334 kB 4.2 MB/s 
     |████████████████████████████████| 48 kB 2.9 MB/s 
     |████████████████████████████████| 250 kB 4.9 MB/s 
     |████████████████████████████████| 27.4 MB 138 kB/s 
     |████████████████████████████████| 715 kB 47.8 MB/s 
     |████████████████████████████████| 206 kB 59.8 MB/s 
     |████████████████████████████████| 131 kB 58.7 MB/s 
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 950 kB 58.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 119 kB 47.4 MB/s 
     |████████████████████████████████| 1.1 MB 46.5 MB/s 
     |████████████████████████████████| 296 kB 67.9 MB/s 
     |████████████████████████████████| 67.3 MB 6.5 kB/s 
     |████████████████████████████████| 2.0 MB 45.8 MB/s 
     |████

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')

In [3]:
train_data.shape


(39073, 15)

In [4]:
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)

In [5]:
train_data.shape


(500, 15)

In [6]:
train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
6118,51,Private,39264,Some-college,10,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,>50K
23204,58,Private,51662,10th,6,Married-civ-spouse,Other-service,Wife,White,Female,0,0,8,United-States,<=50K
29590,40,Private,326310,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,44,United-States,<=50K
18116,37,Private,222450,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,2339,40,El-Salvador,<=50K
33964,62,Private,109190,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,15024,0,40,United-States,>50K


In [7]:
label = 'class'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count        500
unique         2
top        <=50K
freq         365
Name: class, dtype: object


In [8]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass/"
AutoGluon Version:  0.2.0
Train Data Rows:    500
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' >50K', ' <=50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12722.64 MB
	Train

In [9]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States


In [10]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.8397993653393387
Evaluations on test data:
{
    "accuracy": 0.8397993653393387,
    "balanced_accuracy": 0.7437076677780596,
    "mcc": 0.5295565206264157,
    "f1": 0.6242496998799519,
    "precision": 0.7038440714672441,
    "recall": 0.5608283002588438
}


Predictions:  
 0        <=50K
1        <=50K
2         >50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object


In [11]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost,0.842666,0.85,0.111503,0.009649,0.290400,0.111503,0.009649,0.290400,1,True,11
1,RandomForestGini,0.841335,0.84,0.329364,0.110657,0.773932,0.329364,0.110657,0.773932,1,True,5
2,RandomForestEntr,0.840721,0.83,0.324635,0.110944,0.752626,0.324635,0.110944,0.752626,1,True,6
3,LightGBM,0.839799,0.85,0.062109,0.019969,0.235632,0.062109,0.019969,0.235632,1,True,4
4,WeightedEnsemble_L2,0.839799,0.85,0.065294,0.020879,0.667088,0.003185,0.000910,0.431456,2,True,14
5,LightGBMXT,0.839390,0.83,0.049472,0.019633,1.292076,0.049472,0.019633,1.292076,1,True,3
6,CatBoost,0.837957,0.84,0.022577,0.009817,0.847066,0.022577,0.009817,0.847066,1,True,7
7,ExtraTreesEntr,0.834783,0.82,0.331526,0.110670,0.756898,0.331526,0.110670,0.756898,1,True,9
8,ExtraTreesGini,0.834476,0.82,0.327424,0.110593,0.756881,0.327424,0.110593,0.756881,1,True,8
9,LightGBMLarge,0.827823,0.83,0.063226,0.021697,0.372357,0.063226,0.021697,0.372357,1,True,13


In [12]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0               XGBoost       0.85       0.009649  0.290400                0.009649           0.290400            1       True         11
1              LightGBM       0.85       0.019969  0.235632                0.019969           0.235632            1       True          4
2   WeightedEnsemble_L2       0.85       0.020879  0.667088                0.000910           0.431456            2       True         14
3              CatBoost       0.84       0.009817  0.847066                0.009817           0.847066            1       True          7
4      RandomForestGini       0.84       0.110657  0.773932                0.110657           0.773932            1       True          5
5        NeuralNetMXNet       0.84       0.152810  9.377707                0.152810           9.377707        

In [13]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  binary
AutoGluon identified the following types of features:
('category', []) : 8 | ['workclass', 'education', 'marital-status', 'occupation', 'relationship', ...]
('int', [])      : 6 | ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', ...]


In [14]:
time_limit = 60  # for quick demonstration only, you should set this to longest time you are willing to wait (in seconds)
metric = 'roc_auc'  # specify your evaluation metric here
predictor = TabularPredictor(label, eval_metric=metric).fit(train_data, time_limit=time_limit, presets='best_quality')
predictor.leaderboard(test_data, silent=True)

No path specified. Models will be saved in: "AutogluonModels/ag-20210607_152239/"
Presets specified: ['best_quality']
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "AutogluonModels/ag-20210607_152239/"
AutoGluon Version:  0.2.0
Train Data Rows:    500
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' >50K', ' <=50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Us

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L1,0.902783,0.887489,0.121175,0.047899,7.079784,0.121175,0.047899,7.079784,1,True,7
1,LightGBMXT_BAG_L1,0.900161,0.881380,0.481617,0.109193,1.538039,0.481617,0.109193,1.538039,1,True,3
2,WeightedEnsemble_L2,0.896281,0.902263,11.005130,1.604238,50.844822,0.006536,0.001580,1.668572,2,True,14
3,LightGBM_BAG_L1,0.892347,0.866991,0.235506,0.099491,1.337209,0.235506,0.099491,1.337209,1,True,4
4,XGBoost_BAG_L1,0.891681,0.866575,0.582723,0.043422,1.059657,0.582723,0.043422,1.059657,1,True,11
5,RandomForestEntr_BAG_L1,0.888119,0.886301,0.325445,0.119483,0.750771,0.325445,0.119483,0.750771,1,True,6
6,RandomForestGini_BAG_L1,0.886598,0.884698,0.325207,0.116108,0.746056,0.325207,0.116108,0.746056,1,True,5
7,ExtraTreesGini_BAG_L1,0.881065,0.892927,0.330128,0.115510,0.755638,0.330128,0.115510,0.755638,1,True,8
8,ExtraTreesEntr_BAG_L1,0.880851,0.893912,0.330350,0.113281,0.749395,0.330350,0.113281,0.749395,1,True,9
9,NeuralNetFastAI_BAG_L1,0.879856,0.867377,1.358553,0.184843,6.579923,1.358553,0.184843,6.579923,1,True,10
